In [5]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [68]:
class ImdbDownloader:
    def __init__(self,link='https://www.imdb.com/search/title/?title_type=feature,tv_movie&view=simple&sort=num_votes,desc'):
        self.request = requests.get(link)
        self.request.encoding = 'UTF-8'
        self.soup = BeautifulSoup(self.request.text)
        
    def PageLinks(self):
        unique_movies = self.soup.find_all('img', {'class':'loadlate'})#, href=lambda href: href and '/title/' in href)
        links = []
        for movie in unique_movies:
            links.append('https://www.imdb.com/' + movie.parent['href'])
        return links

In [69]:
ImdbDownloader().PageLinks()

['https://www.imdb.com/title//title/tt0111161/',
 'https://www.imdb.com/title//title/tt0468569/',
 'https://www.imdb.com/title//title/tt1375666/',
 'https://www.imdb.com/title//title/tt0137523/',
 'https://www.imdb.com/title//title/tt0110912/',
 'https://www.imdb.com/title//title/tt0109830/',
 'https://www.imdb.com/title//title/tt0133093/',
 'https://www.imdb.com/title//title/tt0120737/',
 'https://www.imdb.com/title//title/tt0167260/',
 'https://www.imdb.com/title//title/tt0068646/',
 'https://www.imdb.com/title//title/tt1345836/',
 'https://www.imdb.com/title//title/tt0167261/',
 'https://www.imdb.com/title//title/tt0816692/',
 'https://www.imdb.com/title//title/tt0114369/',
 'https://www.imdb.com/title//title/tt1853728/',
 'https://www.imdb.com/title//title/tt0172495/',
 'https://www.imdb.com/title//title/tt0372784/',
 'https://www.imdb.com/title//title/tt0848228/',
 'https://www.imdb.com/title//title/tt0102926/',
 'https://www.imdb.com/title//title/tt0361748/',
 'https://www.imdb.c